## 1. Import Libraries

In [1]:
import numpy as np
import cv2 as cv
import argparse
import os
import random
import glob
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
from PIL import Image
import keras
from keras.models import Sequential, Model
from keras.layers import LSTM, Input, Dropout
from keras.layers.convolutional import Conv3D
from tensorflow.keras.layers import ConvLSTM2D
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, MaxPooling3D, UpSampling3D
from tensorflow.keras.preprocessing.image import ImageDataGenerator



### 1.1. Fetch frames sorted by created date

In [2]:

search_dir = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train"
# remove anything from the list that is not a file (directories, symlinks) 
files = list(filter(os.path.isfile, glob.glob(search_dir + "*")))
files = files.sort(key=lambda x: os.path.getmtime(x))
print(files)

None


In [3]:
paths = list(sorted(Path(search_dir).iterdir(), key=os.path.getmtime))
print(paths)

[WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train030'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train031'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train015'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train008'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train027'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train014'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train016'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train013'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train023'), WindowsPath('C:/Users/Gamin

### 1.2. Seperate frames to groups of ten


In [4]:
paths = paths[2:]
path_container = []
n = 10
for path in range(0, len(paths),n):
    chunk  = paths[path:path+n]
print(chunk)
        

[WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train022'), WindowsPath('C:/Users/Gaming PC/Documents/Research/Data/LAD/UCSD_Anomaly_Dataset/UCSDped1/Train/Train001')]


### 1.3. Convert frames to video


In [5]:
# Arguments
dir_path = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001"
# dir_path = chunk
ext = 'tif'
output = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001\\output.mp4"

images = []
for f in os.listdir("C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001"):
    if f.endswith('tif'):
        images.append(f)

# Determine the width and height from the first image
image_path = os.path.join(dir_path, images[0])
frame = cv.imread(image_path)
cv.imshow('video',frame)
height, width, channels = frame.shape
print(images)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv.VideoWriter(output, fourcc, 20.0, (width, height))

for image in images:

    image_path = os.path.join(dir_path, image)
    frame = cv.imread(image_path)

    out.write(frame) # Write out frame to video

    cv.imshow('video',frame)
    if (cv.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
        break

# Release everything if job is finished
out.release()
cv.destroyAllWindows()

print("The output video is {}".format(output))

['001.tif', '002.tif', '003.tif', '004.tif', '005.tif', '006.tif', '007.tif', '008.tif', '009.tif', '010.tif', '011.tif', '012.tif', '013.tif', '014.tif', '015.tif', '016.tif', '017.tif', '018.tif', '019.tif', '020.tif', '021.tif', '022.tif', '023.tif', '024.tif', '025.tif', '026.tif', '027.tif', '028.tif', '029.tif', '030.tif', '031.tif', '032.tif', '033.tif', '034.tif', '035.tif', '036.tif', '037.tif', '038.tif', '039.tif', '040.tif', '041.tif', '042.tif', '043.tif', '044.tif', '045.tif', '046.tif', '047.tif', '048.tif', '049.tif', '050.tif', '051.tif', '052.tif', '053.tif', '054.tif', '055.tif', '056.tif', '057.tif', '058.tif', '059.tif', '060.tif', '061.tif', '062.tif', '063.tif', '064.tif', '065.tif', '066.tif', '067.tif', '068.tif', '069.tif', '070.tif', '071.tif', '072.tif', '073.tif', '074.tif', '075.tif', '076.tif', '077.tif', '078.tif', '079.tif', '080.tif', '081.tif', '082.tif', '083.tif', '084.tif', '085.tif', '086.tif', '087.tif', '088.tif', '089.tif', '090.tif', '091.tif'

### 1.4. Auto-select folder directory


In [6]:
search_dir = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train"
paths = list(sorted(Path(search_dir).iterdir(), key=os.path.getmtime))
print(len(paths), paths[1])
for i in paths:
    print(i)

34 C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train031
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train031
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train015
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train027
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train014
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train016
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train013
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train023
C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\

## 2. Main

### 2.1. Input frames simultaneously and perform Farneback's Algorithm


In [7]:

# Arguments
dir_path = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001"

# Search for all available directories
train_out_path = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\MainTrain"
search_dir = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train"
paths = list(sorted(Path(search_dir).iterdir(), key=os.path.getmtime))
counter_p = 0
counter = 0
for path in paths:
    counter_p =+1
    
    # dir_path = chunk
    ext = 'tif'
    output = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001\\output.mp4"

    images = []
    for f in os.listdir(path):
        if f.endswith('tif'):
            images.append(f)

    # split images into sets of 10
    n = 10
    for frame in range(0, len(images), n):
        frame_chunks = images[frame:frame+n]
        counter = counter+1

        # Determine the width and height from the first image
        image_path = os.path.join(dir_path, images[0])
        frame = cv.imread(image_path)
        cv.imshow('video',frame)
        height, width, channels = frame.shape

        # Define the codec and create VideoWriter object
        fourcc = cv.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
        output_path = os.path.join(path, str(str(counter)+'.mp4'))
        out = cv.VideoWriter(output_path, fourcc, 20.0, (width, height))

        for image in frame_chunks:
            image_path = os.path.join(path, image)
            frame = cv.imread(image_path)

            out.write(frame) # Write out frame to video

            cv.imshow('video',frame)
            if (cv.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
                break

        # Release everything if job is finished
        out.release()
        cv.destroyAllWindows()

        print("The output video is {}".format(output_path))

        # Capture current video
        cap = cv.VideoCapture(cv.samples.findFile(output_path))

        # Read first frame
        ret, frame1 = cap.read()

        # Convert to gray scale
        prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

        # Create mask  to combine calculated optical flow and original video
        hsv = np.zeros_like(frame1)

        # Set image saturation to maximum value as we do not need it
        hsv[..., 1] = 255

        while(cap.isOpened()):
            # Read a frame from video
            ret, frame2 = cap.read()

            if not ret:
                print('No frames grabbed!')
                break

             # Convert new frame format to gray scale
            next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

            # Calculate dense optical flow by Farneback method
            flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

            # Compute the magnitude and angle of the 2D vectors
            mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])

            # Set image hue according to the optical flow direction
            hsv[..., 0] = ang*180/np.pi/2

            # Set image value according to the optical flow magnitude (normalized)
            hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)

            # Convert HSV to RGB (BGR) color representation
            bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

            vector_path = os.path.join(train_out_path, str(str(counter_p)+'i'+str(counter)+'.png'))

            cv.imshow('frame2', bgr)
            cv.imwrite(vector_path, bgr)

            k = cv.waitKey(10) & 0xff
            if k == 27:
                break
            elif k == ord('s'):
                cv.imwrite('C:\\Users\\Gaming PC\\Documents\\Research\\Data\\opticalfb1.png', frame2)
                cv.imwrite('C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001\\opticalhsv1.png', bgr)
            prvs = next

cv.destroyAllWindows()


The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030\1.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030\2.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030\3.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030\4.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030\5.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030\6.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train030\7.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Do

No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008\61.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008\62.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008\63.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008\64.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008\65.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008\66.mp4
No frames grabbed!
The output video is C:\Users\Gaming PC\Documents\Research\Data\LAD\UCSD_Anomaly_Dataset\UCSDped1\Train\Train008\67.mp4
No frames grabbed!
The output vide

KeyboardInterrupt: 

## Set video source

In [ ]:
 cap = cv.VideoCapture(cv.samples.findFile(output))

##  Gunnar Farneback's algorithm - on video


In [ ]:
# Read first frame
ret, frame1 = cap.read()

# Convert to gray scale
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

# Create mask  to combine calculated optical flow and original video
hsv = np.zeros_like(frame1)

# Set image saturation to maximum value as we do not need it
hsv[..., 1] = 255

while(cap.isOpened()):
    # Read a frame from video
    ret, frame2 = cap.read()
    
    if not ret:
        print('No frames grabbed!')
        break

     # Convert new frame format to gray scale
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    
    # Calculate dense optical flow by Farneback method
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Compute the magnitude and angle of the 2D vectors
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    
    # Set image hue according to the optical flow direction
    hsv[..., 0] = ang*180/np.pi/2
    
    # Set image value according to the optical flow magnitude (normalized)
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    
    # Convert HSV to RGB (BGR) color representation
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    
    cv.imshow('frame2', bgr)
    cv.imwrite('C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001\\opticalhsv1.png', bgr)
    k = cv.waitKey(10) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('C:\\Users\\Gaming PC\\Documents\\Research\\Data\\opticalfb1.png', frame2)
        cv.imwrite('C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\Train\\Train001\\opticalhsv1.png', bgr)
    prvs = next

cv.destroyAllWindows()

##  Gunnar Farneback's algorithm - on frames


In [ ]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(cv.samples.findFile("vtest.avi"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next

cv.destroyAllWindows()

### 3.1. LSTM Autoencoder

### 3.1.1. Generators and models

In [8]:
model = Sequential()
#Size of our input images
SIZE = 128

validation_path = "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\MainValidation"

# Define generator for autoencoder training data.
batch_size = 64
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    "C:\\Users\\Gaming PC\\Documents\\Research\\Data\\LAD\\UCSD_Anomaly_Dataset\\UCSDped1\\MainValidation",
    target_size=(SIZE, SIZE),
    batch_size=batch_size,
    class_mode='input'
    )

validation_generator = datagen.flow_from_directory(
    train_out_path,
    target_size=(SIZE, SIZE),
    batch_size=batch_size,
    class_mode='input'
    )

# # Define LSTM training sequence generator
# seq_size = 30  # Number of time steps to look back 

# def to_sequences(x, y, seq_size=1):
#     x_values = []
#     y_values = []

#     for i in range(len(x)-seq_size):
#         #print(i)
#         x_values.append(x.iloc[i:(i+seq_size)].values)
#         y_values.append(y.iloc[i+seq_size])
        
#     return np.array(x_values), np.array(y_values)

# # Generate list of training image paths
# training_dir = train_out_path
# training_paths = list(sorted(Path(training_dir).iterdir(), key=os.path.getmtime))
# print(len(paths), paths[1])

# trainX, trainY = to_sequences(train[['Close']], train['Close'], seq_size)


#Define the autoencoder. 
#Encoder
model = Sequential()
model.add(Conv2D(128, (10, 10), activation='relu', padding='same', input_shape=(8, SIZE, SIZE,3)))
model.add(MaxPooling3D((1, 2, 2), padding='same'))
model.add(Conv2D(64, (6, 6), activation='relu', padding='same'))
model.add(MaxPooling3D((1, 2, 2), padding='same'))
model.add(Conv2D(32, (6, 6), activation='relu', padding='same'))
model.add(MaxPooling3D((1, 2, 2), padding='same'))

# lSTM
model.add(ConvLSTM2D(32, (4, 4), return_sequences=True, padding='same'))
model.add(MaxPooling3D((1, 2, 2), padding='same'))
#model.add(Dropout(rate=0.2))

# model.add(ConvLSTM2D(16, (4, 4), return_sequences=True, padding='same'))
# model.add(UpSampling3D((1, 2, 2)))

# model.add(RepeatVector(1)

model.add(ConvLSTM2D(32, (4, 4
    ), return_sequences=True, padding='same'))
model.add(UpSampling3D((1, 2, 2)))
#model.add(Dropout(rate=0.2))
#model.add(TimeDistributed(Dense(SIZE)))

#Decoder
model.add(Conv2D(32, (6, 6), activation='relu', padding='same'))
model.add(UpSampling3D((1, 2, 2)))
model.add(Conv2D(64, (6, 6), activation='relu', padding='same'))
model.add(UpSampling3D((1, 2, 2)))
model.add(Conv2D(128, (10, 10), activation='relu', padding='same'))
model.add(UpSampling3D((1, 2, 2)))

model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.summary()

#Fit the model. 
history = model.fit(
        train_generator,
        steps_per_epoch= 500 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=75 // batch_size,
        shuffle = True)

Found 200 images belonging to 1 classes.
Found 680 images belonging to 1 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 128, 128, 128)  38528     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 8, 64, 64, 128)   0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 64, 64, 64)     294976    
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 8, 32, 32, 64)    0         
 3D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 32, 32, 32)     73760     
                                      

ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 232, in assert_input_compatibility
        raise ValueError(

    ValueError: Exceptio

In [ ]:
search_dir = train_out_path
paths = list(sorted(Path(search_dir).iterdir(), key=os.path.getmtime))
print(len(paths), paths[1])

### Convolutional Autoencoder and LSTM architecture

In [ ]:
#Define the autoencoder. 
#Encoder
model = Sequential()
model.add(Conv2D(128, (10, 10), activation='relu', padding='same', input_shape=(8, SIZE, SIZE,3)))
model.add(MaxPooling3D((1, 2, 2), padding='same'))
model.add(Conv2D(64, (6, 6), activation='relu', padding='same'))
model.add(MaxPooling3D((1, 2, 2), padding='same'))
model.add(Conv2D(32, (6, 6), activation='relu', padding='same'))
model.add(MaxPooling3D((1, 2, 2), padding='same'))

# lSTM
model.add(ConvLSTM2D(32, (4, 4), return_sequences=True, padding='same'))
model.add(MaxPooling3D((1, 2, 2), padding='same'))
#model.add(Dropout(rate=0.2))

# model.add(ConvLSTM2D(16, (4, 4), return_sequences=True, padding='same'))
# model.add(UpSampling3D((1, 2, 2)))

# model.add(RepeatVector(1)

model.add(ConvLSTM2D(32, (4, 4
    ), return_sequences=True, padding='same'))
model.add(UpSampling3D((1, 2, 2)))
#model.add(Dropout(rate=0.2))
#model.add(TimeDistributed(Dense(SIZE)))

#Decoder
model.add(Conv2D(32, (6, 6), activation='relu', padding='same'))
model.add(UpSampling3D((1, 2, 2)))
model.add(Conv2D(64, (6, 6), activation='relu', padding='same'))
model.add(UpSampling3D((1, 2, 2)))
model.add(Conv2D(128, (10, 10), activation='relu', padding='same'))
model.add(UpSampling3D((1, 2, 2)))

model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.summary()

#Fit the model. 
history = model.fit(
        train_generator,
        steps_per_epoch= 500 // batch_size,
        epochs=1000,
        validation_data=validation_generator,
        validation_steps=75 // batch_size,
        shuffle = True)